IMPORT ALL THE NECESSARY MODULES.

In [10]:
import pandas as pd
import sqlite3 as sql
import pyarrow as pa

EXTRACT ALL THE DATA FROM DATA SOURCES AND CONFIGURE THE SQL CONNECTION.

In [11]:
PRICING = pd.read_csv('data/pricing-1.csv')

TRANSACTIONS = pd.read_csv('data/transactions-1.csv')

USERS = pd.read_csv('data/users-1.csv')

db = sql.connect('master.db')
cursor = db.cursor()


TRANSFORMING THE PRICING DATAFRAME(TABLE) BASE ON THE NORMAL DATA HANDLING PROCESS SUCH AS TYPECASTING AND HANDLING THE POSSIBLE NULL VALUES.  

In [12]:

PRICING['puk'] = PRICING['puk'].fillna('No PUK available').astype(str)
PRICING['product'] = PRICING['product'].fillna('No product name available').astype(str)
PRICING['price'] = PRICING['price'].fillna(0).astype(float)


TRANSFORMING THE TRANSACTIONS DATAFRAME(TABLE) BASE ON THE NORMAL DATA HANDLING PROCESS SUCH AS TYPECASTING, HANDLING THE POSSIBLE NULL VALUES AND CONVERT THE PRODUCT NAME IN TO UPPERCASE AND MANIPULATING THE STRING FOR POSSIBLE ERRORS.

In [13]:


TRANSACTIONS['trans_id'] = TRANSACTIONS['trans_id'].fillna('No trasaction ID available').astype(str)
TRANSACTIONS['user_id'] = TRANSACTIONS['user_id'].fillna('No user ID available').astype(str) 
TRANSACTIONS['product'] = TRANSACTIONS['product'].fillna('No trasaction ID available').astype(str).str.upper() 
TRANSACTIONS['amount'] = TRANSACTIONS['amount'].fillna(0).astype(int)
TRANSACTIONS['trans_date'] = TRANSACTIONS['trans_date'].str.replace('/','-').fillna(pd.Timestamp('1970-01-01')).astype('datetime64[ns]')




TRANSFORMING THE USERS DATAFRAME(TABLE) BASE ON THE NORMAL DATA HANDLING PROCESS SUCH AS TYPECASTING, HANDLING THE POSSIBLE NULL VALUES AND CONVERT EMAILS IN TO LOWERCASE. 

In [14]:
USERS['user_id'] = USERS['user_id'].fillna('No user ID available').astype(str)
USERS['name'] = USERS['name'].fillna('No username available').astype(str)
USERS['email'] = USERS['email'].fillna('No email available').astype(str).str.lower()
USERS['date_joined'] = USERS['date_joined'].fillna('Have not joined yet').astype(str)

TOTAL_SPENT = TRANSACTIONS[['user_id','amount']].groupby('user_id').agg({'amount':'sum'}).reset_index()
USERS = pd.merge(USERS,TOTAL_SPENT,on='user_id',how='inner')
USERS.rename(columns={'amount':'total_spent'}, inplace=True)


LOAD(SAVE) THE 3 DATAFRAME IN MASTER.DB 

I CREATE THE TRANSACTIONS TABLE AS INCREMENTAL MODEL  (CHECK THE PROGRAM BELOW)

In [15]:



table_name = 'TRANSACTIONS'
check = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';"
cursor.execute(check)
table_exists = cursor.fetchone()


if table_exists:

    query = "SELECT * FROM TRANSACTIONS"
    trans_df = pd.read_sql_query(query,db)

    TRANSACTIONS = TRANSACTIONS.loc[TRANSACTIONS['trans_date'] > trans_df['trans_date'].astype('datetime64[ns]').max()]
    TRANSACTIONS.to_sql('TRANSACTIONS',db,index=False, if_exists='append')

else: TRANSACTIONS.to_sql('TRANSACTIONS',db,index=False)         


PRICING.to_sql('PRICING',db,index=False, if_exists='append')
USERS.to_sql('USERS',db,index=False, if_exists='append') 




5

FINALLY LETS CHECK THE QUALITY OF OUR 3 TABLES IN DATABASE BY USING INFO METHOD 

In [16]:

table_name = ['TRANSACTIONS','USERS','PRICING']

for table in table_name: 
  check = f"SELECT * FROM '{table}';"
  df = pd.read_sql_query(check,db)
  df.info()


#PRICING.info()
#TRANSACTIONS.info()
#USERS.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   trans_id    85 non-null     object
 1   user_id     85 non-null     object
 2   product     85 non-null     object
 3   amount      85 non-null     int64 
 4   trans_date  85 non-null     object
dtypes: int64(1), object(4)
memory usage: 3.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      10 non-null     object
 1   name         10 non-null     object
 2   email        10 non-null     object
 3   date_joined  10 non-null     object
 4   total_spent  10 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 528.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column   Non-Null 